In [ ]:
#!pip install opendatasets
#!pip install pandas
import pandas
import tensorflow as tf
import opendatasets as od

In [ ]:
#od.download("https://www.kaggle.com/datasets/aryashah2k/indian-medicinal-leaves-dataset")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
DATADIR = "/content/indian-medicinal-leaves-dataset/Indian Medicinal Leaves Image Datasets/Medicinal plant dataset"
CATEGORIES = ["Neem", "Tulasi", "Amla", "Castor", "Aloevera", "Lemon", "Jasmine", "Insulin", "Pepper", "Bamboo", "Gauva", "Henna", "Mango", "Nagadali", "Hibiscus", "Honge", "Ekka"]
print(len(CATEGORIES))

17


In [ ]:
IMG_SIZE = 256
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

100%|██████████| 146/146 [00:00<00:00, 487.86it/s]

2584


In [ ]:
import random
random.shuffle(training_data)
random.shuffle(training_data)
random.shuffle(training_data)

In [ ]:
X = []
y = []
for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
X = X / 255.0

In [ ]:
with tf.device("/GPU:0"):
  model = Sequential()
  model.add(Conv2D(256, (3, 3), input_shape = (256, 256, 1)))#X.shape[1:]))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))

  model.add(Conv2D(256, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))

  model.add(Conv2D(256, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))

  model.add(Flatten())
  model.add(Dense(64))
  model.add(Activation('relu'))

  model.add(Dense(17))
  model.add(Activation('softmax'))

  model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = 'adam',
              metrics = ['accuracy'])
  earlystopping=tf.keras.callbacks.EarlyStopping(
    monitor='Accuracy',
    min_delta=0.0001,
    patience=2,
    restore_best_weights=True,
    start_from_epoch=5
    )

  tensorboard= tf.keras.callbacks.TensorBoard()
  chkpt = tf.keras.callbacks.ModelCheckpoint(
    'mango.ckpt',
    monitor = 'Accuracy',
    save_best_only = True,

    mode = 'auto',
    save_freq='epoch',
    )

  model.fit(X, y, batch_size = 1, epochs = 15, validation_split = 0.1, callbacks=[earlystopping,tensorboard,chkpt])

Epoch 1/15
2324/2325 [============================>.] - ETA: 0s - loss: 2.8423 - accuracy: 0.0680

2325/2325 [==============================] - 45s 19ms/step - loss: 2.8423 - accuracy: 0.0680 - val_loss: 2.8344 - val_accuracy: 0.0695
Epoch 2/15
2324/2325 [============================>.] - ETA: 0s - loss: 2.8341 - accuracy: 0.0706

2325/2325 [==============================] - 43s 19ms/step - loss: 2.8341 - accuracy: 0.0705 - val_loss: 2.8337 - val_accuracy: 0.0695
Epoch 3/15
2323/2325 [============================>.] - ETA: 0s - loss: 2.8329 - accuracy: 0.0728

2325/2325 [==============================] - 43s 19ms/step - loss: 2.8329 - accuracy: 0.0727 - val_loss: 2.8338 - val_accuracy: 0.0695
Epoch 4/15
2325/2325 [==============================] - ETA: 0s - loss: 2.8328 - accuracy: 0.0727

2325/2325 [==============================] - 43s 19ms/step - loss: 2.8328 - accuracy: 0.0727 - val_loss: 2.8330 - val_accuracy: 0.0695
Epoch 5/15
2323/2325 [============================>.] - ETA: 0s - loss: 2.8328 - accuracy: 0.0728

2325/2325 [==============================] - 42s 18ms/step - loss: 2.8328 - accuracy: 0.0727 - val_loss: 2.8337 - val_accuracy: 0.0695
Epoch 6/15
2325/2325 [==============================] - ETA: 0s - loss: 2.8328 - accuracy: 0.0727

2325/2325 [==============================] - 42s 18ms/step - loss: 2.8328 - accuracy: 0.0727 - val_loss: 2.8333 - val_accuracy: 0.0695
Epoch 7/15
1259/2325 [===============>..............] - ETA: 18s - loss: 2.8318 - accuracy: 0.0707